In [3]:
import os
VERSION = 9
os.environ["CC"] = f"/usr/bin/gcc-{VERSION}"
os.environ["CXX"] = f"/usr/bin/g++-{VERSION}"
import struct
import numpy as np
from tqdm import tqdm
import mediapy

from pathlib import Path
import imageio
import os
import sys
sys.path.append(str(Path(os.path.abspath('')).parent))
print(str(Path(os.path.abspath('')).parent))
import torch
from gaussian_renderer import GaussianModel, render
from gaussian_renderer.eevr import splinerender
from scene import Scene
from scene.cameras import Camera, MiniCam
from torch import nn
import matplotlib.pyplot as plt
import imageio
from pyquaternion import Quaternion
from scene.dataset_readers import ProjectionType

dataset = "london"
# path = "berlin5"
# dataset = "train"
path = "train5"
eval_path = "/home/amai/gaussian-splatting-merge/eval"
# eval_path = "/home/amai/gaussian-splatting-merge/eval"
# output_path = Path("/data/popping_videos/ours0") / dataset / "images"
output_path = Path(f"~/Videos/popping_paths/{path}/ours")

output_path = Path(f"~/Videos/{dataset}/ours/")
output_path.mkdir(parents=True, exist_ok=True)
# f = open(f"/data/video_paths/{dataset}/r1/path.path", "rb")
# f = open(f"/home/amai/Videos/popping_paths/{path}.path", "rb")
f = open(f"/home/amai/Videos/alameda.path", "rb")
# f = open(f"smooth.path", "rb")
data = f.read()
N = int.from_bytes(data[:4])
camera_size = 11

cameras = np.array(struct.unpack(f'>{N*camera_size}f', data[4:])).reshape(N, -1)
full_data = struct.unpack(f'>i{N*camera_size}f', data)
N = full_data[0]
cameras = np.array(full_data[1:]).reshape(N, -1)
print(N)

/home/amai/gaussian-splatting-merge
4018


In [4]:
import shlex
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams

def get_combined_args(args_cmdline):
    cfgfile_string = "Namespace()"

    try:
        cfgfilepath = os.path.join(args_cmdline.model_path, "cfg_args")
        print("Looking for config file in", cfgfilepath)
        with open(cfgfilepath) as cfg_file:
            print("Config file found: {}".format(cfgfilepath))
            cfgfile_string = cfg_file.read()
    except TypeError:
        print("Config file not found at")
        pass
    args_cfgfile = eval(cfgfile_string)

    merged_dict = vars(args_cfgfile).copy()
    for k, v in vars(args_cmdline).items():
        if v != None:
            merged_dict[k] = v
    return Namespace(**merged_dict)


parser = ArgumentParser(description="Testing script parameters")
model = ModelParams(parser, sentinel=True)
pipeline = PipelineParams(parser)
args = parser.parse_args(shlex.split(f"-m {Path(eval_path) / dataset} --images images_2 -r 1"))
print(args.model_path)
args = get_combined_args(args)
model = model.extract(args)
# model.source_path = str(Path("/data/nerf_synthetic") / dataset)
# model.source_path = str(Path("/data/nerf_datasets/tandt/") / dataset)
model.source_path = str(Path("/data/nerf_datasets/zipnerf_ud") / dataset)

model.max_opacity = 0.99

pipeline = pipeline.extract(args)

/home/amai/gaussian-splatting-merge/eval/london
Looking for config file in /home/amai/gaussian-splatting-merge/eval/london/cfg_args
Config file found: /home/amai/gaussian-splatting-merge/eval/london/cfg_args


In [5]:
gaussians = GaussianModel(model.sh_degree, model.max_opacity)
scene = Scene(model, gaussians, load_iteration=-1, shuffle=False)

ic| self.max_opacity: 0.99


Loading trained model at iteration 30000
Reading camera 1874/1874
Loading Training Cameras
Loaded Train Cameras: 1639
Loaded Test Cameras: 235


In [6]:
refcam = scene.getTestCameras()[105]

In [8]:
import cv2

In [9]:
background = torch.tensor([0, 0, 0], dtype=torch.float32, device="cuda")
# width = 1280
# height = 720
# width = 1200
# height = 667
width = refcam.image_width*4
height = refcam.image_height*4
image = torch.ones((3, height, width), dtype=float)
frames = []
for i, smod in tqdm(enumerate(torch.linspace(0.1, 1, 200))):
    with torch.no_grad():
        # rendering = renderer.render(view, pipeline, background)
        rendering = splinerender(refcam, gaussians, pipeline, background, scaling_modifier=smod)["render"]
        byte_rendering = (rendering.permute(1, 2, 0).cpu().numpy()*255).clip(min=0, max=255).astype(np.uint8)
        byte_rendering = cv2.resize(byte_rendering, (refcam.image_width, refcam.image_height), interpolation=cv2.INTER_AREA)
    frames.append(byte_rendering)
    # full_output_path = output_path / f"{i:06d}.png"
    # imageio.imwrite(str(full_output_path), byte_rendering)
    # plt.imshow(byte_rendering)
    # plt.show()

200it [00:12, 15.52it/s]


In [10]:
frames = np.stack(frames, axis=0)
frames = np.concatenate([
    frames,
    torch.as_tensor(frames[-1]).unsqueeze(0).expand(50, -1, -1, -1).numpy(),
    frames[::-1],
])

In [13]:
mediapy.show_video(frames, bps=100000000)
mediapy.write_video("size_animation.mp4", frames, bps=100000000)